# Sessa Emprical Estimator

In [ ]:
Converting the R codes into Python Codes.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime

# Load the dataset (assuming a similar structure to the R script)
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = ["pnr", "eksd", "perday", "ATC", "dur_original"]  # Rename columns to match R script
    df["eksd"] = pd.to_datetime(df["eksd"], format="%m/%d/%Y")  # Convert date format
    return df

# Function to compute the empirical cumulative distribution function (ECDF)
def ecdf(data):
    x = np.sort(data)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

# Function to compute SEE (Sessa Empirical Estimator)
def sessa_empirical_estimator(df, atc_code):
    """
    Computes the duration of filled prescriptions based on k-means clustering.
    """
    # Filter dataset for the selected ATC code
    drug_data = df[df["ATC"] == atc_code].copy()
    
    # Sort by patient ID and prescription date
    drug_data.sort_values(by=["pnr", "eksd"], inplace=True)
    
    # Compute previous prescription date for each patient
    drug_data["prev_eksd"] = drug_data.groupby("pnr")["eksd"].shift(1)
    
    # Drop rows with missing previous prescription date
    drug_data.dropna(subset=["prev_eksd"], inplace=True)
    
    # Compute event intervals (days between prescriptions)
    drug_data["event_interval"] = (drug_data["eksd"] - drug_data["prev_eksd"]).dt.days
    
    # Compute ECDF and retain 80% of the data
    x, y = ecdf(drug_data["event_interval"].values)
    cutoff = np.percentile(x, 80)  # Removing the upper 20%
    filtered_data = drug_data[drug_data["event_interval"] <= cutoff]
    
    # Apply K-Means clustering to classify prescription intervals
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    filtered_data["cluster"] = kmeans.fit_predict(filtered_data[["event_interval"]])
    
    # Compute median prescription duration for each cluster
    cluster_medians = filtered_data.groupby("cluster")["event_interval"].median().reset_index()
    
    # Assign the median duration based on the identified cluster
    merged_data = filtered_data.merge(cluster_medians, on="cluster", suffixes=("", "_median"))
    
    # Assign estimated duration for each prescription
    df = df.merge(merged_data[["pnr", "event_interval_median"]], on="pnr", how="left")
    df["estimated_duration"] = df["event_interval_median"].fillna(df["event_interval_median"].median())
    
    return df

# Example usage:
# df = load_data("med_events.csv")
# result = sessa_empirical_estimator(df, "C09CA01")
# result.head()


ModuleNotFoundError: No module named 'pandas'